# Imports

In [1]:
import pandas as pd
import numpy as np
import torch
import json
import pickle
import gensim
import gensim.downloader as api
import torchtext
from torch import nn, stack, tensor
from google.colab import drive

# Data

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
with open("/content/drive/My Drive/minivqaiust/image_features.pickle", 'rb') as f:
    image_feature = pickle.load(f)

with open("/content/drive/My Drive/minivqaiust/image_question.json") as f:
      image_question_map = json.load(f)

In [4]:
questions={}

for image_id, question in image_question_map.items():
  for q in question:
    questions[q[0]] = {'question_text':q[1], 'image_id': str(image_id)}

In [5]:
#pre proccessing train data
df = pd.read_csv("/content/drive/My Drive/minivqaiust/train.csv")

train_label=torch.tensor(list(df['label']))
train_question=[questions[i]['question_text'] for i in list(df['question_id'])]
train_image_feature = [image_feature[questions[i]['image_id']] for i in list(df['question_id'])]

In [6]:
#pre proccessing validation data
df = pd.read_csv("/content/drive/My Drive/minivqaiust/val.csv")

valid_label=torch.tensor(list(df['label']))
valid_question=[questions[i]['question_text'] for i in list(df['question_id'])]
valid_image_feature = torch.tensor([image_feature[questions[i]['image_id']] for i in list(df['question_id'])])

<ipython-input-6-4e22ac001af4>:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  valid_image_feature = torch.tensor([image_feature[questions[i]['image_id']] for i in list(df['question_id'])])


In [7]:
#pre proccessing test data
df = pd.read_csv("/content/drive/My Drive/minivqaiust/test.csv")

test_question=[questions[i]['question_text'] for i in list(df['question_id'])]
test_image_feature = torch.tensor([image_feature[questions[i]['image_id']] for i in list(df['question_id'])])

# Word Embedding

In [8]:
# Tokenize
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')

# pre embedding model
pre_model = api.load('word2vec-google-news-300')

In [9]:
# Embedding layer
word_embedding = nn.Embedding(len(pre_model.index_to_key) + 1, len(pre_model.get_vector('hi')))

In [10]:
def encode(input):
  return [pre_model.get_index(token, default=-1) + 1 for token in tokenizer(input)]

def padify(input):
  encoded_input = [encode(x) for x in input]
  return stack([nn.functional.pad(tensor(e),(0,15-len(e)),mode='constant',value=0) for e in encoded_input])

In [11]:
# word embedding on train data
with torch.no_grad():
  train_question_embedding = word_embedding(padify(train_question))

In [12]:
# word embedding on validation data
with torch.no_grad():
  valid_question_embedding = word_embedding(padify(valid_question))

In [13]:
# word embedding on test data
with torch.no_grad():
  test_question_embedding = word_embedding(padify(test_question))

# Data loader


In [14]:
train_dataset = torch.utils.data.TensorDataset(train_question_embedding, torch.tensor(train_image_feature), train_label)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

In [15]:
valid_dataset = torch.utils.data.TensorDataset(valid_question_embedding, valid_image_feature, valid_label)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=64, shuffle=True)

# Model

In [16]:
class VQA(nn.Module):
    def __init__(self):
        super(type(self), self).__init__()
        self.lstm = nn.LSTM(256, 512, num_layers=1)
        self.linear = nn.Sequential(
            nn.Linear(8192, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(128),
            nn.Linear(128, 10),
            nn.BatchNorm1d(10),
            nn.ReLU()
        )

    def forward(self, text, image):
        text_features= torch.flatten(self.lstm(text)[0], start_dim=1)
        input = torch.cat([text_features, image], dim=1)
        logits = nn.functional.softmax(self.linear(input), dim=1)
        return logits

In [17]:
model = VQA()

# Train and Validation

In [18]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0004)

In [19]:
def validation_loop(model, dataloader):
  size = len(dataloader.dataset)
  correct = 0
  avg_loss = 0
  for batch, (text, image, y) in enumerate(dataloader):
    prediction = model(text, image)
    loss = loss_function(prediction, y)

    output = [torch.argmax(p).item() for p in prediction]
    correct += (torch.FloatTensor(output) == y).float().sum()
    avg_loss += loss.item()

  accuracy = correct / len(dataloader.dataset)
  return avg_loss, accuracy


In [20]:
def train_loop(dataloader, model, loss_function, optimizer):
    size = len(dataloader.dataset)
    correct = 0
    avg_loss = 0
    for batch, (text, image, label) in enumerate(dataloader):
        prediction = model(text, image)
        loss = loss_function(prediction, label)
        avg_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        output = [torch.argmax(p).item() for p in prediction]
        correct += (torch.FloatTensor(output) == y).float().sum()

    avg_loss /= (len(dataloader.dataset) // 64 + 1)
    accuracy = correct / len(dataloader.dataset)

    val_loss, val_acc = validation_loop(model, valid_dataloader)
    print(f"training / loss: {avg_loss:>7f} | accuracy: {accuracy}")
    print(f"val / loss: {val_loss:>7f} | accuracy: {val_acc}")

In [21]:
del pre_model
del image_feature
del questions
del word_embedding


In [ ]:
for epochs in range(15):
    print(f"Epoch {epochs+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_function, optimizer)

Epoch 1
-------------------------------


# Prediction

In [ ]:
prediction = model(test_question_embedding, torch.tensor(test_image_feature))

output = np.array([torch.argmax(p).item() for p in prediction], dtype='int64')
labeldict = {}
labeldict['question_id'] = list(df['question_id'])
labeldict['label'] = []
for idx, out in enumerate(prediction):
  labeldict['label'].append(int(out))

print(df.head())

In [ ]:
torch.save(model.state_dict(), '/content/drive/My Drive/minivqaiust/minivqa-v1-weights.pth')